# DABN13 - Assignment 3
## Preamble: Forecasting market returns
We continue with the problem of forecasting market return that we illustrated in Lecture 5. In order to train the implementation of PCR/PLS, we will replicate our previous results with a slight twist. More specifically, we will choose tuning parameters via cross-validation. Additionally, the training sets used for model evaluation are defined differently. While the example in Lecture 5 defined an expanding window of training sets with fixed starting period, we are going to use a rolling window that moved both the start and end date of the training set.

The data for this lab is provided by two csv files. 
The file *sorted_portfolios100.csv* contains the monthly returns of 100 equally weighted portfolios sorted by size and the book-to-market ratio. The data is taken from Kenneth French's data library and missing values have been inserted. The time period covered is January 1960 until December 2009
The file *twelve_month_returns.csv* contains 12-month returns on a value-weighted market portfolio. This series takes moving 12-month sums of returns of the U.S. market factor, as provided on Kenneth French's data library. The entry in row $t$ of the dataset corresponds to the market returns over the months $t+1$ until $t+12$. Accordingly, the first observed value in our sample is the 12-month return over the period February 1960 - January 1961. The last observation covers the period January-December 2010.

To begin with the lab, we import both your outcome as well as the 100 predictors into Python using the code below. You might be required to modify the file path.

In [ ]:
import numpy as np
import pandas as pd
from os import chdir

#chdir('??')
portfolios  = pd.read_csv("sorted_portfolios100.csv").iloc[:,1:]
mkt_ret_12m = pd.read_csv("twelve_month_returns.csv").iloc[:,1:] 

mkt_ret_12m_train = mkt_ret_12m.iloc[:540,:]
portfolios_train  = portfolios.iloc[:540,:]

## Part 1: Setting up a PCR pipeline in `sklearn`

Principal component regression is not available in scikit-learn as a distinctive learning algorithm. This is no problem because it is the combination of three simple components:

1. Standardizing inputs,
2. obtaining $k$ principal components from the inputs,
3. running a linear regression of the output on the $k$ principal components (and a constant)

All three operations above are implemented in sklearn. In order to put them togetherm, we will use the very convenient `Pipeline()` functionality.

Pipelines allow us to specify a custom sequence of actions. An existing pipeline can be fit, similar to the specification of a regression model. If we do that, all steps of the pipeline are executed in their specified order.

### Task 1a)

In this task, we will create and fit a pipeline for PCR. First, we need to specify the three actions mentioned above individually. this can be done by using the `StandardizeScalar()`, `PCA()` and `LinearRegression()` functions from several `sklearn` modules. I have already written code below to import these functions. Now, please create three objects `stdz_1a`, `PCA_1a` and `lm_1a` which specify

1. input standardization,
2. PCA with two principal components,
3. a linear regression model,

respectively.

Next, we use the `Pipeline()`-function to define the full sequence of actions that result in PCR. I already prepared the basic structure of the required line of code. All you need is to specify the three steps of our pipeline and to give them suitable names. Please do that and save the resulting object as `pcr_pipe_1a`.

Lastly, we can apply the `fit()`-method to `pcr_spec_1a` to execute the entire pipeline. Do this using `mkt_ret_12m_train`and `portfolios_train` as your output and inputs, respectively. Save the learned model as `pcr_fit_1a`. 

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

stdz_1a = StandardScaler()
PCA_1a  = PCA(n_components=2)
lm_1a   = LinearRegression()

pcr_pipe_1a = Pipeline(steps=[("standardized", stdz_1a), 
                              ("pcs", PCA_1a), 
                              ("regression_model", lm_1a)])
pcr_fit_1a  = pcr_pipe_1a.fit(portfolios_train, mkt_ret_12m_train)

### Task 1b)

We will now get predictions from our learned principal components regression. However, we'll do that time series style. 

When we work with time series, our training data consists of historical series from some starting point in the past until today. Our goal is outcome prediction in the immediate future. That is, our test data may simply be a single data point - at least for the model that we trained at one specific point in time. 

In Task 1a) we used the earliest 540 data points, assuming that this was all our data. Now, we want to get a forecast of data point 541, assuming that this is the immediate future. The more distant future is not of immediate interest for us and we will predict it step-by-step as time passes and more training data appears. 

Now apply the `predict()` method on `pcr_fit_1a` in order to get a prediction from your model for data point 541 in `portfolios_train` and save it as `pcr_pred_1b`.

*Note:* Being able to apply `predict()` to a pipeline is very convenient because we do not have to worry about manual standardization and manual construction of the principal component scores. We simply provide the raw test data point and `sklearn` automatically conducts all necessary transformations. 

In [ ]:
pcr_pred_1b = pcr_fit_1a.predict(portfolios.iloc[540:541], )

print(pcr_pred_1b)

[[0.06281892]]


## Part 2: Model tuning with `sklearn`

Scikit-learn contains very flexible routines for model tuning. The tuning parameter of PCR is the number of principal components to use. We will use the PCR pipeline created in the previous part to choose it via cross-validation.

### Task 2a)

First, we need to define a grid of potential tuning parameter values. This has to be a *dictionary* object in Python which contain arrays with the candidate values for our tuning parameters. The names of the arrays must be identical to the name of the tuning parameter in the corresponding estimator function.

Given that we tune the parameters of an entire pipeline, the names of arrays must also point at which part of the pipeline the tuning parameter can be found. To give an example, assume that want to tune an imaginary parameter `myparam` in a function that included in my pipeline under the name `myfun`. Then, the correspoding array in the dictionary object must have the name `myfun__myparam`. 

Now create a dictionary object `tune_grid_2a` that contains an array with the integers $1,2,3,4,5$ for the number of principal components to obtain in the PCA-step of your pipeline `pcr_pipe_1a`. 

In [ ]:
tune_grid_2a = {"pcs__n_components" : [1, 2, 3, 4, 5]}

### Task 2b)
Next, we must specify how exactly our cross-validation procedure should split the data into training and holdout folds. This is done by specifying a splitter class. 

Since our ultimate goal is to predict market returns in the immediate future, this goal should be mimicked by our cross-validation splits. Hence, we are using the `TimeSeriesSplit()`-function which defines many splits into historical training data and future test data.

Now use `TimeSeriesSplit()` to define an object `cv_splits_2b` which contains a specification of the desired splits. To be specific, we want the following:

- Training folds should always consist of 90 data points.
- The test fold following the training folds should only contain 1 data point
- We want a total of 450 splits, so that the training fold window moves through the entire length of `mkt_ret_12m_train` and `portfolios_train`.
- No gap between training and test folds.

Please check the  `TimeSeriesSplit()`-[documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html#sklearn.model_selection.TimeSeriesSplit) to figure out how to set the function inputs.

In addition to that, apply the `split()`-method to `cv_splits_2b` *and* turn the resulting output into a list object that you save as `cv_splits_list_2b`. Use `portfolios` as the `X`-argument of `split()`.

Lastly, look at the items inside `cv_splits_list_2b`. They contain the indexes of data points that are part of training and hold-out folds for every split. Use the string variable `split_characterization_2b` to characterize the pattern that you see over your 450 splits.

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

# 1.
cv_splits_2b     = TimeSeriesSplit(n_splits = 450, max_train_size = 90, test_size = 1)

# 2.
cv_splits_list_2b = list(cv_splits_2b.split(portfolios))

# 3. 
split_characterization_2b = "The first split contains training data from index 60-149, and tests index 150 \
the next one 61-150 and 151 respectively. It keeps going like this until we reach the final index. This means \
we that we do this 450 times, each time training on 90 observations and testing on 1."

### Task 2c)
Tasks 1a, 2a, and 2b provide us with all the components that we need to specify and to execute a cross-valdation based tuning parameter selection.

First, use the `GridSearchCV()` function in the `model_selection` module of Scikit-learn to specify the setup for model tuning. To be more specific, we want the following:

- We want to tune the model pipeline from Task 1a (don't worry about the number of principal components specified there. It will be overwritten),
- we use the grid of tuning parameter values from Task 2a,
- cross-validation is conducted based on the splits defined in `cv_splits_2b`,
- performance should be evaluated using the (negative) root mean square error.

Look up the `GridSearchCV()`-[documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to figure out how to set the function arguments correctly and save the resulting setup for model tuning as `pcr_tune_2c`.

Second, run the tuning procedure by applying the `fit()`-method to `pcr_tune_2c`. Use `portfolios_train` and `mkt_ret_12m_train` as inputs and output, respectively.


In [ ]:
from sklearn.model_selection import GridSearchCV

# 1.
pcr_tune_2c = GridSearchCV(estimator = pcr_fit_1a, param_grid = tune_grid_2a, cv = cv_splits_2b, scoring = 'neg_root_mean_squared_error', n_jobs=-1)

# 2.
pcr_tune_2c.fit(X=portfolios_train, y=mkt_ret_12m_train)


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=90, n_splits=450, test_size=1),
             estimator=Pipeline(steps=[('standardized', StandardScaler()),
                                       ('pcs', PCA(n_components=2)),
                                       ('regression_model',
                                        LinearRegression())]),
             n_jobs=-1, param_grid={'pcs__n_components': [1, 2, 3, 4, 5]},
             scoring='neg_root_mean_squared_error')

In [ ]:
pcr_tune_2c.cv_results_

{'mean_fit_time': array([0.0100244 , 0.01021403, 0.00965307, 0.00930436, 0.00958696]),
 'std_fit_time': array([0.00600141, 0.0057021 , 0.00616443, 0.00421027, 0.00769241]),
 'mean_score_time': array([0.00215473, 0.00214729, 0.0018829 , 0.00194911, 0.00205927]),
 'std_score_time': array([0.00241505, 0.00224301, 0.00159948, 0.00205517, 0.00209944]),
 'param_pcs__n_components': masked_array(data=[1, 2, 3, 4, 5],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'pcs__n_components': 1},
  {'pcs__n_components': 2},
  {'pcs__n_components': 3},
  {'pcs__n_components': 4},
  {'pcs__n_components': 5}],
 'split0_test_score': array([-0.09836332, -0.10134136, -0.10349464, -0.0957593 , -0.10300666]),
 'split1_test_score': array([-0.11359449, -0.11348349, -0.11534235, -0.10655446, -0.11037733]),
 'split2_test_score': array([-0.13308327, -0.12868585, -0.13041426, -0.12065958, -0.13007213]),
 'split3_test_score': array([-0.14852421

### Task 2d)

`pcr_tune_2c` contains both the optimal tuning parameter value as well as the learned model with this value.  Where inside `pcr_tune_2c` can you find these two objects? Write your specific(!) answer into the string variable `cv_results_2d`.

Additionally, extract the `cv_results_` dictionary from `pcr_tune_2c`, which emerges once we applied the `fit()`-method, into a new object called `pcr_tune_details_2d`. Where inside `pcr_tune_details_2d` can we find the estimated cross-validation estimate of test error for the different tuning parameter candidates? Write your answer into the string variable "test_err_estimates_2d".

In [ ]:
# 1.
cv_results_2d         = "By entering pcr_tune_2c.best_params_ we get the optimal number of pcs among [1,2,3,4,5], which in this\
case is 5, when trained using neg_root_mean_squared_error. We get the learned model by entering pcr_tune_2c.best_estimator_ which is\
Pipeline(steps=[('standardized', StandardScaler()),\
                ('pcs', PCA(n_components=5)),\
                ('regression_model', LinearRegression())])"

# 2.
pcr_tune_details_2d   = pcr_tune_2c.cv_results_
test_err_estimates_2d = "mean_test_score is the key corresponding to the average (over all cvs) test error array, as we can see\
5 pcs has the best score / smallest error"

### Task 2e) 
As an alternative to PCR, we consider PLS for return forecasting. PLS is implemented in `sklearn` in the form of the `PLSRegression()` function. Now do the following:

1. Build a pipeline that first standardizes the inputs and then conducts PLS. In the PLS-part, please do not specify the number of PLS directions, but disable variable scaling. Save the pipeline as `pls_pipe_2e`.
2. Create a tuning parameter grid that contains $1,2,3,4,5$ PLS directions as candidate values. Save it as `tune_grid_2e`
3. Use `GridSearchCV()` to choose the optiimal number of PLS directions. Use the same setup as in Task 2c. Save the resulting object as `pls_tune_2e` and don't forget to fit this object.

In [ ]:
from sklearn.cross_decomposition import PLSRegression

# 1.
pls_pipe_2e  = Pipeline(steps=[("standardized", StandardScaler()), 
                              ("regression_model", PLSRegression(scale=False))])

# 2. 
tune_grid_2e = {"regression_model__n_components" : [1,2,3,4,5]}

pls_tune_2e  = GridSearchCV(estimator = pls_pipe_2e, param_grid = tune_grid_2e, cv = cv_splits_2b, scoring = 'neg_root_mean_squared_error')
pls_tune_2e.fit(X=portfolios_train, y=mkt_ret_12m_train)


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=90, n_splits=450, test_size=1),
             estimator=Pipeline(steps=[('standardized', StandardScaler()),
                                       ('regression_model',
                                        PLSRegression(scale=False))]),
             param_grid={'regression_model__n_components': [1, 2, 3, 4, 5]},
             scoring='neg_root_mean_squared_error')

In [ ]:
pls_pipe_2e.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'standardized', 'regression_model', 'standardized__copy', 'standardized__with_mean', 'standardized__with_std', 'regression_model__copy', 'regression_model__max_iter', 'regression_model__n_components', 'regression_model__scale', 'regression_model__tol'])

In [ ]:
print(pls_tune_2e.best_estimator_)
print(pls_tune_2e.best_params_)
pls_tune_2e.cv_results_

Pipeline(steps=[('standardized', StandardScaler()),
                ('regression_model',
                 PLSRegression(n_components=5, scale=False))])
{'regression_model__n_components': 5}


{'mean_fit_time': array([0.0028739 , 0.00287054, 0.00297679, 0.00306621, 0.0031212 ]),
 'std_fit_time': array([0.00025707, 0.00019485, 0.0001829 , 0.00018073, 0.00016393]),
 'mean_score_time': array([0.00134322, 0.00130386, 0.00131647, 0.00131716, 0.00129826]),
 'std_score_time': array([1.24167569e-04, 8.86977571e-05, 1.04476595e-04, 9.35955697e-05,
        8.01941135e-05]),
 'param_regression_model__n_components': masked_array(data=[1, 2, 3, 4, 5],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'regression_model__n_components': 1},
  {'regression_model__n_components': 2},
  {'regression_model__n_components': 3},
  {'regression_model__n_components': 4},
  {'regression_model__n_components': 5}],
 'split0_test_score': array([-0.11645192, -0.09614957, -0.09654333, -0.10907492, -0.12392824]),
 'split1_test_score': array([-0.12649177, -0.08265173, -0.06696856, -0.06043958, -0.05714063]),
 'split2_test_score': array([-

### Task 2f)
Which tuning parameter choices do we get for PCR and PLS? Write your answer into the string variable `tune_best_2g`. Additionally, save the learned models with best tuning parameters as `pcr_fit_tuned_2f` and `pls_fit_tuned_2f`, respectively.

In [ ]:
tune_best_2g = "In this case we get the same result, that 5 is the best option. So 5 both for PCR and PLS"
pcr_fit_tuned_2f = pcr_tune_2c.best_estimator_
pls_fit_tuned_2f = pls_tune_2e.best_estimator_

## Part 3: Comparing two candidate algorithms

Part 2 helped us to find the optimal tuning parameter values for both PCR and PLS. However, which of these two (tuned) algorithms is better for forecasting returns? We will find out in this rather difficult part.

In order to choose between PCR and PLS, we check how well either of the two procedures predicts the outcomes of data points $541, 542, \ldots, 600$. We still do this in time-series fashion. That means we use data up to data point 540 to predict data point 541 and move start and end of training and hold-out data to the end of the sample one data point at a time. 
The data used to train our model is still the 90 data points prior to the test data point.

### Task 3a)

Create a vector `mkt_ret_12m_test` and a matrix `portfolios_test` that we can use for performance evaluation on the last 60 data points of `mkt_ret_12m` and `portfolios`. That is, it must go back long enough in time to include the training data for an output prediction of data point 541.

In [ ]:
mkt_ret_12m_test = mkt_ret_12m.iloc[450:,:]
portfolios_test  = portfolios.iloc[450:,:]

### Task 3b)

Use `cross_validate()` in sklearn to evaluate the performance of PCR and PLS with the tuning parameter values that you arrived at in Task 2f. `cross_validate()` conducts cross-validation with specified data and a desired scheme for splitting it without any grid search.

First, use `TimeSeriesSplit()`to specify splits into training and holdout folds that contain 90 and 1 data points, respectively. Save this specification as `cv_splits_3a`. The training window should move through the data points in the test data created in Task 3a one data point at a time. The holdout fold of the first split should be the 91st observation. The holdout fold of the last split should be the very last observation in the test data.

Second, use `cross_validate()` to estimate the test error of your tuned PCR and PLS algorithms from Task 2f and use root mean squared error to measure accuracy. Save the resulting objects as `pcr_eval_3b` and `pls_eval_3b`, respectively. Check the `cross_val_score()`-[documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) to get the function inputs right. Save your performance evaluation results as `pcr_eval_3b` and `pls_eval_3b`. 

Third, check which object inside `pcr_eval_3b` or `pcr_eval_3b` contains the root mean squared errors of all holdout folds. Write your answer into the string variable `where_is_rmse_3b`. Since we only have one data point in every holdout fold, the RMSE is identical to the absolute value of our prediction error on the test fold.



In [ ]:
from sklearn.model_selection import cross_validate 

# 1.
cv_splits_3b = TimeSeriesSplit(n_splits = 60, max_train_size = 90, test_size = 1)

# 2.
pcr_eval_3b = cross_validate(pcr_fit_tuned_2f, portfolios_test, mkt_ret_12m_test, scoring = "neg_root_mean_squared_error", cv = cv_splits_3b)

pls_eval_3b = cross_validate(pls_fit_tuned_2f, portfolios_test, mkt_ret_12m_test, scoring = "neg_root_mean_squared_error", cv = cv_splits_3b)

where_is_rmse_3b = "test_score, which is an array"

In [ ]:
pcr_eval_3b['test_score']

array([-3.70973713e-05, -2.10424406e-02, -9.55765568e-03, -5.61210305e-02,
       -2.18138365e-02, -1.61124181e-01, -7.56539631e-02, -3.79324348e-02,
       -2.98607540e-02, -3.02627791e-02, -1.83758016e-02, -2.51326575e-02,
       -3.70306541e-03, -7.26392912e-03, -1.76212220e-02, -2.46205994e-04,
       -6.69921964e-02, -5.78866803e-02, -6.41361042e-02, -5.22334088e-02,
       -8.14181439e-02, -6.90947315e-02, -2.56339222e-03, -1.58033022e-02,
       -7.93105089e-02, -8.09104702e-02, -9.75802857e-02, -5.99736440e-02,
       -4.48372231e-02, -1.62601046e-01, -6.82465575e-02, -5.41643014e-02,
       -1.67571590e-01, -3.73673405e-01, -3.86904004e-01, -3.48064952e-01,
       -3.91819721e-01, -4.63198982e-01, -3.48458855e-01, -2.27025032e-01,
       -1.36856701e-01, -3.78899880e-01, -2.00707973e-01, -1.93418992e-01,
       -2.37900749e-01, -2.81829289e-01, -2.04110797e-01, -1.37646708e-01,
       -5.57247923e-02, -6.63068483e-02, -6.17826806e-02, -2.89620909e-02,
       -8.85531523e-02, -

### Task 3c)

The RMSE provided by `cross_validate()` is hard to interpret if we want to know whether any of the two machine learning algorithms is practically useful at all. A better alternative is out-of-sample R-squared. We define it mathematically as 
$$
  R^2 = 1 - \sum_{t=t_0}^{T} \frac{(y_t - \hat{y}_t)^2}{(y_t -\bar{y}_{t,-90})^2}.
$$
Here, 

- $y_t$ is the observed output at time $t$, 
- $t_0$ is the time period of the earliest test observation (541 in our case),
- $\hat{y}_t$ is the model prediction of $y_t$ at time $t$,
- $\bar{y}_{t,-90}=90^{-1}\sum_{\ell}^{90} y_{t-\ell}$ is the average outcome of the 90 output values before $t$

To construct this R-squared, do the following

1. Construct a vector `ymeans_3c` whose element $t$ contains the average of `mkt.ret.12m` from time point $450+t$ to $539+t$. You could apply the `rolling()` method to `mkt_ret_12m_test` or write a for-loop to achieve that.
2. Extract the sum of squared prediction error of PLS and PCR over all test folds from `pcr_eval_3b` amd `pls_eval_3b`. Save them as numbers `pcr_avg_errsq_3c` and `pls_avg_errsq_3c`, respectively
4. Construct the summed squared prediction error of the historical 90-day average output but using `mkt_ret_12m_test` and `ymeans_3c`. Save the resulting number as `histavg_avg_errsq_3c`.
5. Construct out-of-sample R2 for PCR and PLS using the objects created in the previous two steps. Save them as `pcr_R2_oos_3c` and `pls_R2_oos_3c`, respectively.

Does PCR outperform PLS or is it the other way around? Does R-squared suggest that any of the two methods is useful for predicting returns? Motivate your conclusion and express it in the string variable `conclusion_3c`. 

In [ ]:
positive_rmse_values_pcr = -np.array(pcr_eval_3b['test_score'])
positive_rmse_values_pls = -np.array(pls_eval_3b['test_score'])

# 1.
ymeans_3c            = mkt_ret_12m_test.rolling(window=90).mean()

# 2.
pcr_sum_errsq_3c = np.sum(positive_rmse_values_pcr ** 2)
pls_sum_errsq_3c = np.sum(positive_rmse_values_pls ** 2)

# 3.
histavg_sum_errsq_3c = np.sum(np.square(mkt_ret_12m_test[-60:] - ymeans_3c[-60:]))

# 4.
pcr_R2_oos_3c        = 1 - (pcr_sum_errsq_3c / histavg_sum_errsq_3c)
pls_R2_oos_3c        = 1 - (pls_sum_errsq_3c / histavg_sum_errsq_3c)



#conclusion_3c = "??"

x    0.517056
dtype: float64
x    0.807294
dtype: float64


### Part 4: PCR manually

In this advanced part, we will repeat Part 1 without using the `pca()`-transformation in scikit-learn. More specifically, we will replace it with matrix operations in Numpy.

### Task 4a)

By convention, principal components are obtained from standardized data. In order to do that, we use the `StandardScaler()` function from in scikit-learn that we already made part of the PCR pipeline. 
Statistical standardization using `StandardScaler()` needs to be first specified and then fit on data. Then it can be used to transform any desired dataset. We will do this now:

1. Use `StandardScaler()` to specify and to fit an object that contains sample means and variances of each variable in `portfolios_train`. Save this object as `stdz_X_4a`.
2. Apply the `transform()`-method to `stdz_X_4a` to standardize `portfolios_train`. Save the resulting data frame as a two-dimensional *Numpy array* `X_train_stdz_4a`. 
3. An alternative method for variable scaling would be normalization. Search among the functions [in the preprocessing module](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) of `sklearn` for an equivalent to `StandardScaler()`. Then, conduct the previous two steps using this function. However, when you save objects, use the name `norm_X_4a` instead of `stdz_X_4a` and `X_train_norm_4a` instead of `X_train_stdz_4a`. 

In [ ]:
# 1.
stdz_X_4a       = StandardScaler()

# 2.
stdz_X_4a.fit(portfolios_train)

X_train_stdz_4a = stdz_X_4a.transform(portfolios_train)

# 3.
from sklearn.preprocessing import MinMaxScaler
norm_X_4a       = MinMaxScaler()

norm_X_4a.fit(portfolios_train)

X_train_norm_4a = norm_X_4a.transform(portfolios_train)

### Task 4b)

We can now use the eigendecomposition (a.k.a spectral decomposition) to obtain principal components (PCs). The `eig()`-function in the linear algebra module of NumPy  allows us to obtain a list object containing the eigenvectors and eigenvalues of any square matrix that we feed into `eig()`. Now, we want to construct the scores of the first two principal components. In order to do that, review the slides of Lecture 5 and do the following:

1. Use `eig()` to get the loadings of the first two principal components of `X_train_stdz_4a`. Save them as `PCloadings_4b`.
2. Use `PCloadings_4b` to construct the scores of the first two PCs of `X_train_stdz_4a` and save them as `PCscores_4b`.

In [ ]:
# 1.
from numpy.linalg import eig
eigdecomp_4b  = eig(np.dot(X_train_stdz_4a.T, X_train_stdz_4a)) 
PCloadings_4b = eigdecomp_4b[1][:,0:2]

# 2.
PCscores_4b = np.dot(X_train_stdz_4a,PCloadings_4b)

Proceed with the second step of PCR: 

1. Create an input matrix `Z_4c` containing `PCscores_4b` and a constant. 
2. Obtain the learned coefficients for a linear regression model with inputs `Z.4c`, output `mkt_ret_12m_train` and squared error loss. Do this manually using only the `solve()` and `transpose()` commands in NumPy as well as matrix operations. Save the result as `pcr_coefs_4c`.

In [ ]:
# 1.
constant_column = np.ones((PCscores_4b.shape[0], 1))
Z_4c         = np.hstack((PCscores_4b, constant_column))

# 2.
pcr_coefs_4c = np.dot(np.linalg.inv(np.dot(np.transpose(Z_4c), Z_4c)), np.dot(np.transpose(Z_4c), mkt_ret_12m_train))

### Task 4d)

Now that we have learned our PCR model, we can get an output prediction for data point 541.

1. Transform the 100 portfolio returns in the 451-st data point of `portfolios` using your fitted standardization object from Task 4a. In order to prevent Python from converting this single row of `portfolios` into a Pandas series, you might add *additional* square brackets around the row that you index. Save the resulting $1\times 100$ NumPy array of standardized inputs as `X_test_4d`.
2. Use `PCloadings_4b` to get the PC scores corresponding to the 100 values in `X_test_4d`. Save them, together with a constant, in a $1\times 3$ NumPy array `Z_test_4d`.
3. Get a prediction from the model learned in Task 4c for the input combination `Z_test_4d`. Save this prediction as `pcr_pred_4d`.

In [ ]:
# 1.
X_test_4d = stdz_X_4a.transform(portfolios.iloc[540:541]) #OBS! We think you made a typo writing 451 instead of 541

# 2. 
PCscores_4d = np.dot(X_test_4d, PCloadings_4b)
Z_test_4d = np.array([PCscores_4d[0][0], PCscores_4d[0][1], 1])

# 3. 
pcr_pred_4d = np.dot(np.transpose(pcr_coefs_4c), Z_test_4d)

[[0.06281893]]
[0.06281893]
